In [5]:
import pandas as pd 
path = r"D:\DeepFinance\data\interim\news\concatenated_news_filtered.parquet"
df = pd.read_parquet(path)
print(df.keys())

Index(['author', 'content', 'datetime', 'source', 'summary', 'title', 'url',
       'date', 'equity', 'extracted_summary'],
      dtype='object')


In [21]:
path = r"D:\DeepFinance\data\processed\unified_dataset_test.pkl"
import pickle as pkl 
with open(path, 'rb') as f: 
    data = pkl.load(f)

date_with_news = 0 
for d in data: 
    if data[d]['news']:
        date_with_news += 1
print(date_with_news)
    

288


In [23]:
import pickle
import os
import numpy as np
import pandas as pd
from configs.config import GlobalConfig

def inspect_unified_dataset(filename='unified_dataset_test.pkl'):
    file_path = os.path.join(GlobalConfig.PROCESSED_PATH, filename)
    
    if not os.path.exists(file_path):
        print(f"❌ File not found: {file_path}")
        return

    print(f"📂 Loading {file_path}...")
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
    
    # 1. Basic Stats
    dates = sorted(list(data.keys()))
    print("\n" + "="*40)
    print("📊 DATASET STATISTICS")
    print("="*40)
    print(f"Total Trading Days: {len(dates)}")
    if dates:
        print(f"Start Date: {dates[0]}")
        print(f"End Date:   {dates[-1]}")
    
    # 2. Coverage Check
    days_with_news = sum(1 for d in data if data[d].get('news'))
    days_with_embed = sum(1 for d in data if data[d].get('news_embedding'))
    days_with_filing_q = sum(1 for d in data if data[d].get('filing_q'))
    
    print(f"\nDays with Price/Macro:   {len(dates)} (100%)")
    print(f"Days with News:          {days_with_news} ({days_with_news/len(dates)*100:.1f}%)")
    print(f"Days with News Embed:    {days_with_embed} ({days_with_embed/len(dates)*100:.1f}%)")
    print(f"Days with Filings (10Q): {days_with_filing_q} ({days_with_filing_q/len(dates)*100:.1f}%)")

    # 3. Detailed Inspection of a Sample Day
    # Chọn ngày ở giữa để đảm bảo không dính ngày đầu/cuối thiếu dữ liệu
    sample_date = dates[len(dates)//2] 
    sample_data = data[sample_date]
    
    print("\n" + "="*40)
    print(f"🔍 INSPECTING SAMPLE DATE: {sample_date}")
    print("="*40)
    
    # --- Price ---
    print("\n[1] Market Price:")
    if sample_data.get('price'):
        for ticker, val in list(sample_data['price'].items())[:3]: # Show max 3 tickers
            print(f"   - {ticker}: {val}")
    else:
        print("   ❌ Missing Price Data")

    # --- Macro ---
    print("\n[2] Macro Indicators:")
    if sample_data.get('macro'):
        print(f"   - Keys: {list(sample_data['macro'].keys())}")
        print(f"   - Sample (us10y): {sample_data['macro'].get('us10y', 'N/A')}")
    else:
        print("   ❌ Missing Macro Data")

    # --- News ---
    print("\n[3] News Data:")
    news_data = sample_data.get('news', {})
    if news_data:
        tickers_with_news = list(news_data.keys())
        print(f"   - Tickers with news: {tickers_with_news}")
        
        # Check structure of first news item
        first_ticker = tickers_with_news[0]
        first_news_item = news_data[first_ticker][0]
        print(f"   - Example ({first_ticker}):")
        print(f"     Title: {first_news_item.get('title', 'N/A')[:50]}...")
        print(f"     Keys available: {list(first_news_item.keys())}")
    else:
        print("   ⚠️ No news for this date")

    # --- Embedding ---
    print("\n[4] News Embeddings:")
    embed_data = sample_data.get('news_embedding', {})
    if embed_data:
        tickers_with_emb = list(embed_data.keys())
        print(f"   - Tickers with embeddings: {tickers_with_emb}")
        # Check vector shape
        first_vec = embed_data[tickers_with_emb[0]]
        print(f"   - Vector Type: {type(first_vec)}")
        if isinstance(first_vec, (list, np.ndarray)):
            print(f"   - Vector Dimension: {len(first_vec)}")
            print(f"   - Sample values: {first_vec[:5]}...")
    else:
        print("   ⚠️ No embeddings for this date")

    print("\n" + "="*40)
    print("✅ INSPECTION COMPLETE")
    print("="*40)

if __name__ == "__main__":
    inspect_unified_dataset()

📂 Loading d:\DeepFinance\data\processed\unified_dataset_test.pkl...

📊 DATASET STATISTICS
Total Trading Days: 288
Start Date: 2024-05-01
End Date:   2025-06-25

Days with Price/Macro:   288 (100%)
Days with News:          288 (100.0%)
Days with News Embed:    288 (100.0%)
Days with Filings (10Q): 0 (0.0%)

🔍 INSPECTING SAMPLE DATE: 2024-11-25

[1] Market Price:
   - TSLA: {'open': Ticker
TSLA    360.140015
Name: 144, dtype: float64, 'high': Ticker
TSLA    361.929993
Name: 144, dtype: float64, 'close': Ticker
TSLA    338.589996
Name: 144, dtype: float64}
   - AMZN: {'open': Ticker
AMZN    199.279999
Name: 144, dtype: float64, 'high': Ticker
AMZN    201.949997
Name: 144, dtype: float64, 'close': Ticker
AMZN    201.449997
Name: 144, dtype: float64}
   - MSFT: {'open': Ticker
MSFT    415.316296
Name: 144, dtype: float64, 'high': Ticker
MSFT    417.996506
Name: 144, dtype: float64, 'close': Ticker
MSFT    415.723297
Name: 144, dtype: float64}

[2] Macro Indicators:
   - Keys: ['vix', 'yield